In [1]:
import os
path=r"E:\科研\ESV计算\Sri Lanka\attributeTable"
tab=os.listdir(path)
tables=[i for i in tab if i.endswith("t.csv")]

In [2]:
tables

['2017t.csv', '2018t.csv', '2019t.csv', '2020t.csv', '2021t.csv', '2022t.csv']

In [3]:
values=r"E:\科研\ESV计算\Sri Lanka\ESV_Pro.csv"

In [4]:
import pandas as pd

In [5]:
v=pd.read_csv(values)
v.drop(v.columns[0],axis=1)

,Unnamed: 0,食物生产,原料生产,水资源供给,气体调节,气候调节,净化环境,水文调节,水土保持土壤保持,维持养分循环,生物多样性,景观美学
0,水系,12888.800,3705.53,133560.190,12405.470,36894.19,89416.050,1647188.640,14983.230,1127.770,41083.0500,30449.790
1,冰川积雪,0.000,0.00,34799.760,2899.980,8699.94,2577.760,114871.430,0.000,0.000,161.1100,1449.990
2,草原,1611.100,2255.54,1288.880,8216.610,21588.74,7088.840,15788.780,9988.820,805.550,9022.1600,4027.750
3,耕地(旱地),13694.350,6444.40,322.220,10794.370,5799.96,1611.100,4349.970,16594.330,1933.320,2094.4300,966.660
4,湿地,8216.610,8055.50,41727.490,30610.900,57999.60,57999.600,390369.530,37216.410,2899.980,126793.5700,76205.030
5,裸地,0.000,0.00,0.000,322.220,0.00,1611.100,483.330,322.220,0.000,322.2200,161.110
6,森林(针+针阔+阔叶)//3,4398.303,4833.30,5268.297,33398.103,99888.20,29048.133,62236.793,40648.053,3109.423,37002.1337,16223.777


In [6]:
mapTable={
    1:"水系",
    2:"森林(针+针阔+阔叶)//3",
    4:"湿地",
    5:"耕地(旱地)",
    7:"城镇用地",
    8:"裸地",
    9:"冰川积雪",
    10:"云层",
    11:"草原"
}

In [7]:
from collections import defaultdict

In [17]:
AT=[]

for i in range(len(tables)):
    newT=defaultdict(int)
    year=tables[i].split('.')[0]
    tb=pd.read_csv(os.path.join(path,tables[i]))
    for i in range(len(tb)):
        if (k:=mapTable[tb["Value"][i]]) in v["Unnamed: 0"].unique():
            _=tb['Count'][i]/100
            for n,g in enumerate(v.columns[2:].tolist(),2):
                newT[g]+=_/1e10*v[v["Unnamed: 0"]==k].iloc[0,n]
    AT.append(newT)

In [18]:
d=pd.DataFrame(AT,index=["%s"%i for i in range(2017,2023)])

In [19]:
d

,食物生产,原料生产,水资源供给,气体调节,气候调节,净化环境,水文调节,水土保持土壤保持,维持养分循环,生物多样性,景观美学
2017,3.163530,2.639121,4.164629,14.341230,40.214782,13.048185,49.957827,17.745527,1.418044,15.747460,7.164717
2018,3.213560,2.610514,4.231160,13.824042,38.446334,12.623490,50.868888,17.144965,1.377816,15.100190,6.891699
2019,3.280067,2.606669,4.446675,13.599994,37.621122,12.537051,53.525603,16.893757,1.362485,14.862786,6.815638
2020,3.395818,2.633864,4.599190,13.555920,37.276836,12.532223,55.383854,16.872263,1.366420,14.781022,6.799869
2021,3.546387,2.715064,4.912846,13.999621,38.454788,13.029339,59.077424,17.434939,1.412987,15.357597,7.094531
2022,3.473253,2.676975,4.843067,13.816264,37.987200,12.873438,58.224817,17.197832,1.392523,15.188147,7.017108


In [20]:
d.iloc[0,:].sum()

169.60505094475744

In [21]:
newD=pd.DataFrame([],columns=["供给服务","调节服务","支持服务","文化服务"])

In [22]:
import numpy as np

In [23]:
newD["供给服务"]=d.iloc[:,0]+d.iloc[:,1]+d.iloc[:,2]
x=d.iloc[:,3]
for i in range(4,7):
    x+=d.iloc[:,i]
newD["调节服务"]=x
x=d.iloc[:,7]
for i in range(8,10):
    x+=d.iloc[:,i]
newD['支持服务']=x
newD['文化服务']=d.iloc[:,-1]

In [24]:
newD

,供给服务,调节服务,支持服务,文化服务
2017,9.967279,117.562023,34.911031,7.164717
2018,10.055234,115.762754,33.622971,6.891699
2019,10.333411,117.283769,33.119027,6.815638
2020,10.628872,118.748833,33.019704,6.799869
2021,11.174298,124.561173,34.205523,7.094531
2022,10.993295,122.901719,33.778502,7.017108


In [25]:
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.globals import CurrentConfig
import pyecharts.faker as F
from pyecharts.faker import Faker

In [26]:
a=Bar()
a.add_xaxis([str(i) for i in range(2017,2023)])
for i in range(4):
    a.add_yaxis(newD.columns[i],newD.iloc[:,i].tolist())
a.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
a.set_global_opts(title_opts=opts.TitleOpts(title='斯里兰卡生态服务价值变化',subtitle="单位：Billion Rupee"),xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15)),
                 )
a.render_notebook()

In [27]:
d_sum=pd.DataFrame([],columns=['sum'])
d_sum['sum']=newD.iloc[:,0]
for i in range(1,4):
    d_sum['sum']+=newD.iloc[:,i]
d_sum

,sum
2017,169.605051
2018,166.332658
2019,167.551846
2020,169.197278
2021,177.035524
2022,174.690624


In [28]:
l=Line()
l.add_xaxis([str(i) for i in range(2017,2023)])
l.add_yaxis('Total',d_sum.iloc[:,0])
l.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
l.set_global_opts(title_opts=opts.TitleOpts(title='斯里兰卡生态服务价值变化',subtitle="单位：Billion Rupee"),xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15)),
                 )
l.render_notebook()

In [29]:
a.overlap(l).render_notebook()

In [30]:
a=Bar()
a.add_xaxis(list(d.columns))
for i in range(2017,2023):
    a.add_yaxis(str(i),d.loc[str(i)].tolist())
a.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
a.set_global_opts(title_opts=opts.TitleOpts(title='斯里兰卡生态服务价值变化',subtitle="单位：Billion Rupee"),xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-15)),
                 )
a.render_notebook()

In [34]:
y=d.iloc[-1,:]
dic=dict(zip(d.columns,y.tolist()))

In [35]:
d

,食物生产,原料生产,水资源供给,气体调节,气候调节,净化环境,水文调节,水土保持土壤保持,维持养分循环,生物多样性,景观美学
2017,3.163530,2.639121,4.164629,117.562023,40.214782,13.048185,49.957827,34.911031,1.418044,15.747460,7.164717
2018,3.213560,2.610514,4.231160,115.762754,38.446334,12.623490,50.868888,33.622971,1.377816,15.100190,6.891699
2019,3.280067,2.606669,4.446675,117.283769,37.621122,12.537051,53.525603,33.119027,1.362485,14.862786,6.815638
2020,3.395818,2.633864,4.599190,118.748833,37.276836,12.532223,55.383854,33.019704,1.366420,14.781022,6.799869
2021,3.546387,2.715064,4.912846,124.561173,38.454788,13.029339,59.077424,34.205523,1.412987,15.357597,7.094531
2022,3.473253,2.676975,4.843067,122.901719,37.987200,12.873438,58.224817,33.778502,1.392523,15.188147,7.017108


In [36]:
dic

{'食物生产': 3.4732528902365942,
 '原料生产': 2.6769751902025902,
 '水资源供给': 4.843066858688555,
 '气体调节': 122.90171933838893,
 '气候调节': 37.98719990479867,
 '净化环境': 12.873438231808406,
 '水文调节': 58.22481700966074,
 '水土保持土壤保持': 33.778502140155894,
 '维持养分循环': 1.392523217095374,
 '生物多样性': 15.188147297182738,
 '景观美学': 7.017107701083367}

In [37]:
pie=Pie()
pie.add("",[[i,j] for i,j in dic.items()],
        label_opts=opts.LabelOpts(position="left",formatter="{d}%"),radius=["30%","50%"])
pie.set_global_opts(
    title_opts=opts.TitleOpts(title="斯里兰卡 2022 ESV分项",pos_left="40%"),
    legend_opts=opts.LegendOpts(
        type_="scroll",
        pos_top="20%",
        pos_left="80%",
        orient="vertical"
    )
)
pie.render_notebook()